<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Kaggle Dataset into Google Drive, so that CoLab can assess these data.

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [3]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 94.6MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 62.7MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 67.4MB/s]
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 126MB/s]


In [4]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip


Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv',index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape

((59400, 39), (14358, 39), (59400, 2), (14358, 2))

## Begin project by establishing Baseline metrics.
### Method 1: Majority baseline by value_counts. Random chance predictions have 
1. 54% functional well pump.
2. 38% non-functional pump.
3. 7%  repairable.

In [7]:
df_label.status_group.value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [0]:
accuracy = df_label.status_group.value_counts(normalize=True)[0]

# Create score dataframe
score=pd.DataFrame(columns=['method','accuracy', 'shape'])
score.loc[0]=['majority_baseline',accuracy, (1,1)]

### Method 2: Feed LogisticRegression with numeric only.
* Numeric data value can range so wide that no regression iteration can ever be enough to converge. 
* Do not increase max_iteration of LogisticRegression, instead call a Scaler.

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline

df_num = df_train.select_dtypes(np.number)

X=df_num
y = df_label.status_group

pipe = make_pipeline(
    RobustScaler(),
    LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=500)
    )

pipe.fit(X,y)

y_pred = pipe.predict(X)

accuracy= accuracy_score(y, y_pred)
score.loc[score.shape[0]]=['logReg_num', accuracy, X.shape]

print('Accuracy score=', accuracy)

Accuracy score= 0.5577946127946128


### Method 2: LogisticRegression with non-numeric categorical value object only.
* Encoders are friend of LogisticRegression, transforming non-numeric data  into numeric.
#### But nasty data character are hidden landmine bomb to Encoder. 
* Nasty datatype - NaN, '?', space in front/after, keyword:False, True, None .



In [0]:
def obj_cleanup(X5):
  X5.replace('\ 2\ ','two', regex=True, inplace=True)
  X5.replace(';','semicolon', regex=True, inplace=True)
  X5.replace("`", "back_quote", regex=True, inplace=True)
  X5.replace("'", "quote", regex=True, inplace=True)
  X5.replace("\(", "left_paren", regex=True, inplace=True)
  X5.replace("\)", "right_paren", regex=True, inplace=True)
  X5.replace("\/", "back_slash", regex=True, inplace=True)
  X5.replace("\\", "fwd_slash", regex=True, inplace=True)
  X5.replace("   ", "space", regex=True, inplace=True)
  X5.replace("\.", "dot", regex=True, inplace=True)
  return X5

In [0]:
# Example of finding problem maker
'''test = df.select_dtypes([np.object]).apply(lambda x: x.str.contains(';').any())
str_cols = df.select_dtypes([np.object]).columns
str_cols[test]'''

#df_test.loc[df_test['scheme_name'].str.contains('\.')]['scheme_name']

In [0]:
# Best define a function for data cleanup, so that the cleanup process can be
# applied to df_test data as well.

def cleanup(data):
  data.fillna('unknown', axis=1, inplace=True)
  
  # Nasty keyword(False, True) object breaks OneHotEncoder
  data['permit'] = data['permit'].astype(dtype='str')
  data['public_meeting'] = data['public_meeting'].astype(dtype='str')
  
  # One unique value adds no value, hence removed
  data.drop('recorded_by', axis=1, inplace=True)
  
  return data

In [11]:
# Example of nasty keyword data value hidden in permit column out 39 column features.
df_train.permit.unique()

array([False, True, nan], dtype=object)

In [12]:
# copy()- make sure all cleanup work are copied over.
df_train = cleanup(df_train).copy()
df_test = cleanup(df_test).copy()

df_train.permit.unique()

array(['False', 'True', 'unknown'], dtype=object)

In [13]:
# Any NaN will show as True
df_train.isnull().any().any()

False

#### High cardinality features add no value but noise to regression.  
* Filter out features with high number of unique categorical values by eyeballing method.
* nunique() function lists out number of unique categorical values.


In [0]:
# Exclude non-number columns with high number of categorical values. 
# High cardinality columns when split by encoding turns into many useless noisy data.
filter_cols  = df_train.select_dtypes(object).nunique().sort_values().index.tolist()[:-6]
df_obj = df_train[filter_cols].copy()

#### OneHotEncoding non-numeric categorical value into 0 and 1.
* Automatically increase both training accuracy score and testing bias into the model.

In [15]:
from sklearn.preprocessing import OneHotEncoder

X=df_obj
y=df_label.status_group

pipe = make_pipeline(
    OneHotEncoder(categories='auto'),
    LogisticRegression(solver='lbfgs', multi_class='ovr',
                      max_iter=500))

pipe.fit(X,y)
y_pred = pipe.predict(X)
accuracy=accuracy_score(y, y_pred)

score.loc[score.shape[0]] = ['logReg_OneHotEncoder', accuracy, X.shape]
print('Accuracy score=', accuracy)

Accuracy score= 0.757020202020202


### Method 3: LogisticRegression with categorical value without Encoder.
* Transforming unique categorical value into nominal value is more accurate and efficient than OneHotEncoding.

In [16]:
df_cat = df_obj.apply(lambda x: x.astype('category').cat.codes)

X=df_cat
y=df_label.status_group

pipe = make_pipeline(
    RobustScaler(),
    LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=500)
    )

pipe.fit(X,y)

y_pred = pipe.predict(X)

accuracy=accuracy_score(y, y_pred)
score.loc[score.shape[0]] = ['logReg_cat_codes', accuracy, X.shape]
print('Accuracy score=', accuracy)

Accuracy score= 0.6286531986531987


### Method 4: Include all non-numeric categorical value (all cardinality) using cat code.
* Converting non-numeric to category code, solve the high cardinality columns problem.

In [17]:
df_cat_all = df_train.apply(lambda x: x.astype('category').cat.codes)
X=df_cat_all
y=df_label.status_group

pipe = make_pipeline(
    RobustScaler(),
    LogisticRegression(solver='lbfgs',multi_class='auto',max_iter=500)
    )

pipe.fit(X,y)

y_pred = pipe.predict(X)

accuracy=accuracy_score(y, y_pred)
score.loc[score.shape[0]] = ['logReg_cat_allcodes', accuracy, X.shape]
print('Accuracy score=', accuracy)

Accuracy score= 0.6557912457912458


### Method 5: Everything both numeric and non-numeric using Encoder
* OneHotEncoder going crazy transforming both numeric and non-numeric, turning LogisticRegression into a neural network overfitting model?
#### Accuracy of 96% on training data? OneHotEncoder not to be trusted. Using this model to predict on df_test data, yields nonsense category unknown error.

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

#X=df_train
y=df_label.status_group

pipe = make_pipeline(
    OneHotEncoder(categories='auto'),
    LogisticRegression(solver='lbfgs', multi_class='ovr',
                      max_iter=2000))

pipe.fit(X,y)
y_pred = pipe.predict(X)
accuracy=accuracy_score(y, y_pred)

score.loc[score.shape[0]] = ['logReg_allnumcat_Encoding', accuracy, X.shape]
print('Accuracy score=', accuracy)

Accuracy score= 0.9674579124579125


In [19]:
score

,method,accuracy,shape
0,majority_baseline,0.543081,"(1, 1)"
1,logReg_num,0.557795,"(59400, 9)"
2,logReg_OneHotEncoder,0.757020,"(59400, 23)"
3,logReg_cat_codes,0.628653,"(59400, 23)"
4,logReg_cat_allcodes,0.655791,"(59400, 38)"
5,logReg_allnumcat_Encoding,0.967458,"(59400, 38)"


## Data Exploration - making sense of data

In [0]:
# grouping longitude+latitude into local centroid

from sklearn.cluster import KMeans

def add_local_gps(X3):
  points=X3[['longitude','latitude']]
  points = points.drop(points[points.longitude == 0].index)
  
  km = KMeans(n_clusters=20)
  km = km.fit(points)
  
  points['centroid']=km.labels_
  return points['centroid']

In [0]:
# Plot longitude+latitude with centroid
from sklearn.cluster import KMeans

def viz_gps(X3):
  points=X3[['longitude','latitude']]
  points = points.drop(points[points.longitude == 0].index)

  '''
  sum_of_squared_distanced = []

  K = range(1,25)
  for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(points)
    sum_of_squared_distanced.append(km.inertia_)

  plt.plot(K, sum_of_squared_distanced, marker="X")
  plt.xlabel('k', fontsize=15)
  plt.ylabel('Sum_of_squared_distanced', fontsize=15)
  plt.title('Elbow Method to find the optimal k by eyeball', fontsize=15)
  plt.show()
'''
  km = KMeans(n_clusters=20)
  km = km.fit(points)

  plt.scatter(points.longitude, points.latitude, c=km.labels_, cmap='rainbow')

  plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], color='black')

In [0]:
# feature engineering-turning continuous numeric into discrete value.
# Hopefully logistic classifier can map discrete value to target
def num_cleanup(X1):
  '''
  date_recorded
  ['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year']
  '''
  
  # categorical value nan converted to unknown to be processed by regression.
  X2 = X1.fillna('unknown')
  
  # added recorded_year_built
  # convert date_recorded as year 
  dates = pd.to_datetime(X2.date_recorded)
  X2['recorded_year_built'] = dates.dt.year - X2['construction_year']
  X2.loc[X2.recorded_year_built > 1000 ] = 0
  X2.date_recorded = dates.dt.year
  
  #
  bins=[-100,0,100,500,1000,1250,1500,2000]
  labels=[0,100,500,1000,1250,1500,2000]
  #X2['gps_height_bin']= pd.cut(X2.gps_height, bins, labels=labels)
  #X2.drop(['gps_height'], axis=1, inplace=True)
  
  # replace longitude, latitude with centroid
  X2['centroid']=add_local_gps(X1)
  #X2.drop(['longitude','latitude'], axis=1, inplace=True)
  
  type_dict = {'amount_tsh':'float64',    # nunique()=98
               'date_recorded':'float64', # date into year
               'gps_height':'float64',
               #'gps_height_bin':'float64',    #added bins; reduce score 0.02
               'centroid':'float64',  # centroid=25,2000: no impact on score.
               'num_private':'float64',   # nunique()=65
               'population':'float64',    # nunique()=1049
               'construction_year':'float64',   # nunique()=55
               'recorded_year_built':'float64', #added
               'region_code':'float64',   # nunique()=27
               'district_code':'float64'  # nunique=20 
              }
  X2 = X2.astype(dtype = type_dict)
  
  X2 = X2.fillna(0)

  return X2

In [0]:
import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.03):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
df_train.funder = df_train.funder.replace('0', 'missing')
X_dummies = pd.get_dummies(df_train.funder)
funder_filter=variance_threshold_selector(X_dummies)

In [0]:
df_train.funder = df_train.funder.replace('0', 'missing')
X_dummies = ce.OneHotEncoder().fit_transform(s)
dummy_filter=variance_threshold_selector(X_dummies)

In [0]:
def obj_cleanup(X):
  
  X4=X.copy()
  
  '''# extraction_type_class 0.714 - 0.713
  X4.extraction_type_class.replace('other','otr_extr',inplace=True)
    
  # management 0.713 - 0.708
  X4.management=X4.management.replace('unknown','unkwn_mgmt')
  X4.management=X4.management.replace('other','otr_mgmt')
   
  # payment
  X4.payment=X4.payment.replace('unknown','unkwn_pmt')
  
  # quality_group
  X4.quality_group=X4.quality_group.replace('unknown','unkwn_qual_grp')
    
  # quantity
  X4.quantity=X4.quantity.replace('unknown','unknown_quantity')
  
  # waterpoint_type
  X4.waterpoint_type=X4.waterpoint_type.replace('other','otr_wtr')
  '''
  type_dict = {
               'basin':'object',   
               'region':'object', 
               'lga':'object',
               'extraction_type_class':'object', 
               'management':'object',  
               'payment':'object',   
               'quality_group':'object',
               'quantity':'object' ,
               'source_type':'object', 
               'waterpoint_type':'object'   
              }
  cols=list(type_dict.keys())
  X4 = X4[cols].astype(dtype = type_dict)
  
  
  return X4
  

In [29]:
from sklearn.metrics import accuracy_score

X=df_final
y = df_label.status_group

pipe.fit(X,y)

y_pred = pipe.predict(X)
print('Accuracy score=', accuracy_score(y, y_pred))



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Accuracy score= 0.725


In [0]:
y_test_pred = pipe.predict(df_final_test)

In [0]:
df_obj_clean.apply(lambda x: x.astype('category').cat.codes)

In [12]:
# Clean and prepare data
df_num = df_train.select_dtypes('number').copy()
df_num['date_recorded'] = df_train['date_recorded']
df_num_test = df_test.select_dtypes('number').copy()
df_num_test['date_recorded'] = df_test['date_recorded']

df_clean = num_cleanup(df_num).copy()
df_clean_test = num_cleanup(df_num_test).copy()


df_obj = df_train.select_dtypes(object).copy()
df_obj_test = df_test.select_dtypes(object).copy()

df_obj_clean = obj_cleanup(df_obj).copy()
df_obj_test_clean = obj_cleanup(df_obj_test).copy()

df_final = df_clean.join(df_obj_clean)
df_final_test = df_clean_test.join(df_obj_clean)

df_final.shape, df_final_test.shape


((59400, 22), (14358, 22))

In [0]:
y_test_pred = pipe.predict(t)

In [0]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warning

warnings.filterwarnings(action='ignore', category='FutureWarning')

X=df_num
y = df_label.status_group

pipe = make_pipeline(
    SelectKBest(f_classif),
    LogisticRegression(max_iter=500))

param_grid = {
    'selectkbest__k': [5,9, 'all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [.1, 1.0, 10.0]
}


gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)

gs.fit(X, y)


print('Accuracy Score:', gs.best_score_)
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head(3)

In [0]:
%%time
from sklearn.metrics import accuracy_score

X=df_num
y = df_label.status_group

pipe.fit(X,y)

y_pred = pipe.predict(X)
print('Accuracy score=', accuracy_score(y, y_pred))

y_test_pred = pipe.predict(df_num_test)

In [0]:
y_test_pred = pipe.predict(df_num_test)

In [0]:
scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None, max_iter=1500), 
                        X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

In [0]:
# Xgboost

from xgboost import XGBClassifier
modelxgb = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 3, maximize = False, eval_metric = 'merror', eta = .1,
                      max_depth = 14, colsample_bytree = .4)

In [0]:
# Xgboost
from sklearn.model_selection import cross_val_score

score = (cross_val_score(modelxgb, X, y, cv=5,n_jobs = -1))
score.mean()

In [0]:
df_num.drop(['gps_height_bin','amount_tsh','date_recorded','num_private',
            'gps_height','construction_year'], axis=1, inplace=True)

In [0]:
df_num.drop(['region_code'],axis=1, inplace=True)

In [26]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

X=df_num.copy()
y = df_label.status_group

model = LogisticRegression(solver='lbfgs', 
             multi_class='auto',max_iter=3000).fit(X, y)
y_pred = model.predict(X)

print('Accuracy score=', accuracy_score(y, y_pred))

Accuracy score= 0.5985353535353536
CPU times: user 1min 2s, sys: 33.7 s, total: 1min 35s
Wall time: 48.2 s


In [0]:
y_pred = model.predict(df_num_test)

In [0]:
#submission as format as df_sample
y_submission = pd.DataFrame()

y_submission['id'] = df_test.index
y_submission['status_group']=y_pred
y_submission.shape

In [0]:
from google.colab import files

y_submission.to_csv('data_all.csv', index=False)
files.download("data_all.csv")